In [1]:
%pip install gurobipy

  Using cached gurobipy-9.1.2-cp38-cp38-macosx_10_11_x86_64.whl (4.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from itertools import product
from math import sqrt
import xlrd

import gurobipy as gp
from gurobipy import GRB

In [3]:
df = pd.read_excel('Price_Cost.xlsx', sheet_name='feedstock')

In [4]:
p=df.iloc[:,1]

In [26]:
def excel2matrix(path,page):
    data = xlrd.open_workbook(path)
    table = data.sheets()[page]
    nrows = table.nrows  
    ncols = table.ncols
    datamatrix = np.zeros((nrows, ncols))
    for i in range(ncols):
        cols = table.col_values(i)
        datamatrix[:, i] = cols
    return datamatrix

In [30]:
theta_fra =excel2matrix('Yield.xls',0) 

In [32]:
fra_u_c = pd.read_excel('Utility_Chemical.xlsx', sheet_name='fra')

In [36]:
fra_u_c.iloc[:,0]

0            Kraft
1          Sulfite
2             Soda
3     Aq. Alkaline
4             AFEX
5              AAP
6              ARP
7           FT-DAP
8              CAH
9              DAH
10          GVL-AH
11           IL-AH
12             AMP
13          FT-HWP
14             SEP
15      Organosolv
16     FA-assisted
17             RCF
18    ILD&Ionosolv
19      Mechanical
20             CEL
21            EMAL
Name: Unnamed: 0, dtype: object

In [5]:
num_b=2
num_fra=2
u1=[1,1]

In [6]:
prod_b_fra=list(product(range(num_b),range(num_fra)))

In [8]:
m = gp.Model('lignin_valorization')

I_b = m.addVars(num_b, vtype=GRB.BINARY, name='Select1')
F_b_fra = m.addVars(prod_b_fra, vtype=GRB.CONTINUOUS, name='Flow1')
Fin_b_fra = m.addVars(prod_b_fra, vtype=GRB.CONTINUOUS, name='Inflow1')

Academic license - for non-commercial use only - expires 2022-08-19
Using license file C:\Users\yzw301.PSU\gurobi.lic


In [21]:
for p in range(num_fra):
        m.addConstr(Fin_b_fra[(b,p)] == gp.quicksum(F_b_fra[(b,p)] * I_b[b] for b in range(num_b)))

In [22]:
U_b_fra = m.addVars(prod_b_fra, vtype=GRB.CONTINUOUS, name='Utility1')

In [24]:
for b in range(num_b):
    for p in range(num_fra):
        m.addConstr(U_b_fra[(b,p)] == u1[p] * Fin_b_fra[(b,p)])

In [26]:
Q_b=10

In [27]:
for b in range(num_b):
    for p in range(num_fra):
        m.addConstr(F_b_fra[(b,p)] <= I_b[b] * Q_b)

In [28]:
Fout_fra = m.addVars(prod_b_fra, vtype=GRB.CONTINUOUS, name='Chemical1')

In [29]:
C_b_fra = m.addVars(prod_b_fra, vtype=GRB.CONTINUOUS, name='Chemical1')

In [30]:
theta_fra = np.array([[1,1],[1,1]])

In [32]:
for b in range(num_b):
    for p in range(num_fra):
        m.addConstr(Fout_fra[(b,p)] == theta_fra[b,p] * (Fin_b_fra[(b,p)] + C_b_fra[(b,p)]) )

In [33]:
num_dpl=2
num_ip1=2

In [34]:
prod_fra_dpl = list( product( range(num_b), range(num_fra), range(num_dpl), range(num_ip1) ) )

In [35]:
F_fra_dpl = m.addVars(prod_fra_dpl, vtype=GRB.CONTINUOUS, name='Flow2')

In [36]:
y_ip1_fra = np.array([[1,0],[0,1]])

In [43]:
I_dpl = m.addVars(num_b, vtype=GRB.BINARY, name='Select2')

In [44]:
I_fra = m.addVars(num_b, vtype=GRB.BINARY, name='Select2')

In [72]:
I_upg = m.addVars(num_b, vtype=GRB.BINARY, name='Select2')

In [75]:
for b in range(num_b):
    for fra in range(num_fra):
        for ip1 in range(num_ip1):
            m.addConstr( y_ip1_fra[ip1,fra]*Fout_fra[(b,fra)] == gp.quicksum(F_fra_dpl[(b,fra,dpl,ip1)]*I_dpl[dpl] for dpl in range(num_dpl)))

In [46]:
prodin_fra_dpl=list(product(range(num_b), range(num_ip1), range(num_dpl)))

In [47]:
Fin_fra_dpl = m.addVars(prodin_fra_dpl, vtype=GRB.CONTINUOUS, name='Inflow2')

In [49]:
for b in range(num_b):
    for p in range(num_dpl):
        for ip1 in range(num_ip1):
            m.addConstr(Fin_fra_dpl[(b,ip1,p)] == gp.quicksum(F_fra_dpl[(b,fra,p,ip1)]*I_fra[fra] for fra in range(num_fra)) )

In [50]:
u_dpl=[1,1]

In [51]:
U_dpl = m.addVars(prodin_fra_dpl, vtype=GRB.CONTINUOUS, name='Utility2')

In [52]:
for b in range(num_b):
    for p in range(num_dpl):
        for ip1 in range(num_ip1):
            m.addConstr(U_dpl[(b,ip1,p)] == u_dpl[p] * Fin_fra_dpl[(b,ip1,p)])

In [53]:
C_dpl = m.addVars(prodin_fra_dpl, vtype=GRB.CONTINUOUS, name='Chemical2')
c_dpl=[1,1]

In [54]:
for b in range(num_b):
    for p in range(num_dpl):
        for ip1 in range(num_ip1):
            m.addConstr(C_dpl[(b,ip1,p)] == c_dpl[p] * Fin_fra_dpl[(b,ip1,p)])

In [55]:
Fout_dpl = m.addVars(prodin_fra_dpl, vtype=GRB.CONTINUOUS, name='Main2')

In [56]:
for b in range(num_b):
    for p in range(num_fra):
        for ip1 in range(num_ip1):
            m.addConstr(Fout_dpl[(b,ip1,p)] == theta_fra[ip1,p] * (Fin_fra_dpl[(b,ip1,p)] + C_dpl[(b,ip1,p)]))

In [57]:
W_dpl = m.addVars(prodin_fra_dpl, vtype=GRB.CONTINUOUS, name='Waste2')

In [60]:
phi_dpl=np.array([[1,0],[0,1]])

In [62]:
for b in range(num_b):
    for p in range(num_dpl):
        for ip1 in range(num_ip1):
            m.addConstr(W_dpl[(b,ip1,p)] == (1- theta_fra[ip1,p] - phi_dpl[ip1,p]) * (Fin_fra_dpl[(b,ip1,p)] + C_dpl[(b,ip1,p)]))

In [66]:
num_ip2=2
num_upg=2

In [68]:
y_ip2_dpl = np.array([[1,0],[0,1]])

In [70]:
F_dpl_upg = m.addVars(prod_fra_dpl, vtype=GRB.CONTINUOUS, name='Flow3')

In [76]:
for b in range(num_b):
    for dpl in range(num_dpl):
        for ip2 in range(num_ip2):
            #for ip1 in range(num_ip1):
                m.addConstr(y_ip2_dpl[ip2,dpl] *Fout_dpl[(b,ip1,p)] == gp.quicksum(F_dpl_upg[(b,dpl,upg,ip2)] * I_upg[upg] for upg in range(num_upg)) )

In [78]:
prodin_dpl_upg = list(product(range(num_b), range(num_ip2), range(num_upg)))

In [79]:
Fin_dpl_upg = m.addVars(prodin_dpl_upg, vtype=GRB.CONTINUOUS, name='Inflow3')

In [80]:
for b in range(num_b):
    for p in range(num_upg):
        for ip2 in range(num_ip2):
            m.addConstr(Fin_fra_dpl[(b,ip2,p)] == gp.quicksum(F_dpl_upg[(b,dpl,p,ip2)]*I_dpl[dpl] for dpl in range(num_dpl)) )

In [83]:
m.setObjective(I_b.prod([1,1,1]), GRB.MINIMIZE)

In [84]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 52 rows, 120 columns and 120 nonzeros
Model fingerprint: 0x7a4321e7
Model has 81 quadratic constraints
Variable types: 108 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [68]:
a = [1, 2, 3] ; b = [4, 5, 6]; c = [7, 8, 9]

In [74]:
w2 = np.array( [ [a,b,c], [a,b,c] ] )

In [75]:
print(w2)

[[[1 2 3]
  [4 5 6]
  [7 8 9]]

 [[1 2 3]
  [4 5 6]
  [7 8 9]]]


In [78]:
w2[0,1]

array([4, 5, 6])

In [3]:
list(product(range(2), range(2)))

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [5]:
print(list(product(range(2), range(2))))

[(0, 0), (0, 1), (1, 0), (1, 1)]


In [15]:
y=list(product(range(2), range(2)))

In [16]:
print(y)

[(0, 0), (0, 1), (1, 0), (1, 1)]


In [18]:
A = [[1, 4, 5, 12], 
    [-5, 8, 9, 0],
    [-6, 7, 11, 19]]

In [19]:
print(A)

[[1, 4, 5, 12], [-5, 8, 9, 0], [-6, 7, 11, 19]]


In [20]:
A[1]

[-5, 8, 9, 0]

In [21]:
A[0][0]

1

In [25]:
a1=np.array([[1,2,3],[4,5,6]])

In [26]:
print(a1)

[[1 2 3]
 [4 5 6]]


In [31]:
a1[0,0]

1

In [4]:
a=np.array([[1,1,1],[0,0,0]])

In [5]:
a[1,1]

0

In [7]:
list(product(range(2),range(2)))

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [19]:
b=np.array([[[1,1,1],[0,0,0]],[[0,1,1],[0,0,0]]])

In [20]:
b[1]

array([[0, 1, 1],
       [0, 0, 0]])

In [24]:
a=[1,2,3]

In [25]:
len(a)

3

In [26]:
num_a=len(a)

In [27]:
num_a

3

In [29]:
a[1]

2

In [30]:
a

[1, 2, 3]

In [32]:
range(num_a)

range(0, 3)

In [33]:
for i in range(num_a):
    print(i)

0
1
2
